# ML Pipeline Preparation

### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
import nltk
nltk.download(['punkt','wordnet','stopwords'])

import pandas as pd
from sqlalchemy import create_engine
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re
import pickle

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import classification_report

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Mathijn\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Mathijn\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Mathijn\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Load data from database

In [2]:

engine = create_engine('sqlite:///..\data\DisasterResponse.db')
df = pd.read_sql_table('messages', engine) 
X = df.message
Y = df.iloc[:,4:]

### 2. Tokenization function to process the text data

In [3]:
def tokenize(message):    

    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+' 
          
    # Removing any URLs
    detected_urls = re.findall(url_regex, message)

    for url in detected_urls:
        message = message.replace(url, "urlplaceholder")

    # Removing punctuation
    message = re.sub(r"[^a-zA-Z0-9]", " ", message) 

    # Tokenizing the message
    tokens = word_tokenize(message)           

    # Lemmatize, lowercase, strip and also removing stopwords    
    clean_tokens = [WordNetLemmatizer().lemmatize(t).lower().strip() for t in tokens if t not in stopwords.words("english")]

    return(clean_tokens)
    

### 3. Build a machine learning pipeline
This machine pipeline takea in the `message` column as input and output classification results on the other 36 categories in the dataset. The countvectorizer calls the tokenizer function defined above. The transformed data is passed to the TfidTransformer and finally the data is fitted with a random forest classifier. The multi output classifier is used because the categories are not mutually exlusive: more labels can exist for each message.

In [4]:
pipeline = Pipeline([

    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier())) 

])

First, I am doing a randomized search of a parameter grid in order to keep computation time down. 

In [5]:
def random_cv(pipeline):
    
    parameters = {   
        'vect__ngram_range': [(1,1),(1,2),(2,2)],     
        'vect__max_df': (0.25, 0.5, 0.75, 1),
        'vect__max_features': (None, 2000, 4000, 6000, 8000, 10000),
        'tfidf__use_idf': (True, False),
        'clf__estimator__n_estimators': [10, 40, 70, 100, 130, 160, 190, 230],
        'clf__estimator__max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
        'clf__estimator__max_features': ['auto', 'sqrt'], 
        'clf__estimator__min_samples_leaf': [1, 2, 4],
        'clf__estimator__min_samples_split': [2, 5, 10],
        'clf__estimator__bootstrap': [True, False]
    }

    cv = RandomizedSearchCV(pipeline, 
            param_distributions = parameters,
            scoring = ['f1_micro','precision_micro','recall_micro'],
            refit ='f1_micro',
            verbose = 3,
            n_jobs = 4,
            n_iter = 100      
            )

    return(cv)


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

First randomized search is run

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, 
    test_size=0.2, 
    random_state=42)

model = random_cv(pipeline)
model.fit(X_train, y_train) 


Fitting 5 folds for each of 100 candidates, totalling 500 fits
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed: 32.8min
[Parallel(n_jobs=4)]: Done 120 tasks      | elapsed: 126.6min
[Parallel(n_jobs=4)]: Done 280 tasks      | elapsed: 314.8min
[Parallel(n_jobs=4)]: Done 500 out of 500 | elapsed: 557.7min finished


### 5. Test your model
The precision, recall, and F1 score of the best model of the random search are shown for each of the labels

In [20]:
y_pred = model.predict(X_test)

print(classification_report(y_test,y_pred,target_names=Y.columns))

precision    recall  f1-score   support

               related       0.85      0.93      0.89      3998
               request       0.83      0.49      0.62       891
                 offer       0.00      0.00      0.00        24
           aid_related       0.74      0.71      0.73      2164
          medical_help       0.63      0.16      0.26       435
      medical_products       0.75      0.20      0.31       279
     search_and_rescue       0.57      0.15      0.24       136
              security       0.12      0.01      0.02        96
              military       0.52      0.10      0.17       158
                 water       0.82      0.61      0.70       335
                  food       0.84      0.79      0.81       584
               shelter       0.78      0.52      0.62       468
              clothing       0.71      0.17      0.28        70
                 money       0.80      0.07      0.13       112
        missing_people       0.00      0.00      0.00        63

Below the best fitting parameters

In [21]:
model.best_params_

{'vect__ngram_range': (1, 2),
 'vect__max_features': 4000,
 'vect__max_df': 0.75,
 'tfidf__use_idf': True,
 'clf__estimator__n_estimators': 100,
 'clf__estimator__min_samples_split': 5,
 'clf__estimator__min_samples_leaf': 1,
 'clf__estimator__max_features': 'sqrt',
 'clf__estimator__max_depth': None,
 'clf__estimator__bootstrap': True}

### 6. Improve your model
Use grid search to find better parameters. Here we are finetuning the best parameters found by the randomsearch to see if we can improve model fit further.

In [6]:
def grid_cv(pipeline):
    
    parameters = {   
        'vect__ngram_range': [(1,2)],     
        'vect__max_df': (0.7, 0.75, 0.8),
        'vect__max_features': (3000, 4000, 5000),
        'clf__estimator__n_estimators': [100],
        'clf__estimator__max_features': ["sqrt"], 
        'clf__estimator__min_samples_split': [5] 
    }

    cv = GridSearchCV(pipeline,
        param_grid = parameters,
        scoring = ['f1_micro','precision_micro','recall_micro'],
        refit ='f1_micro',
        verbose = 3,
        n_jobs = 4                  
        )

    return(cv)


In [9]:
model = grid_cv(pipeline)
model.fit(X_train, y_train) 

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


PicklingError: Could not pickle the task to send it to the workers.

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

In [ ]:
y_pred = model.predict(X_test)

print(classification_report(y_test,y_pred,target_names=Y.columns))

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [ ]:
pickle.dump(model, open('models\model.p', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.